# Using Predictive Analysis To Predict Melbourne House Price  

By Niladri Ghosh

## 1. Identify Problem

Real Estate has became one of the main places to put your investment in recent times. With the current instability in the market it has became too risky to invest in stock markets, gold prices constanly going down and goverment not considering crypocurrency as a serious option. That being said, owing a property seems to be a viable option to put your money in. So with this motivation we need to analyze what fectors leads to the price of a property in an area, we'll be predicting house prices in Melbourne, Australia.

### 1.1 Expected Outcome

Given house price data from Domain.com.au (and other sources) provides various features of a particular property like number of rooms, address, suburb ,etc. Since this is a __regression problem__, our __final output would be the price of the house estimated by our model__.

### 1.2 Objective

Since many features in our data is categorical value, we need to find appropriate techniques to convert them into integers, as our model only inputs numerical values and finally predict house prices. In machine learning this is a regression problem.

> Thus, the goal is to estimate the price of various houses provided the different unique features of the property. To acheive this we have used machine learning regression methods to fit a function that can predict the continuos values (price) for the new inputs.

### 1.3 Identify Data Sources

The [Melbourne House Price](https://www.kaggle.com/anthonypino/melbourne-housing-market?select=Melbourne_housing_FULL.csv) dataset is available at Kaggle by [Tony Pino](https://www.kaggle.com/anthonypino). The dataset contains 34,857 rows of data and 21 columns (20 features and 1 taget).

* The 5th column provides the price of the house.
* Columns 1-4 and 6-21 provides various details for the given house.

__Getting Started : Load libraries and set options__

In [1]:
# load libraries
import numpy as np
import pandas as pd

__Load Dataset__

First load the supplied CSV file using Pandas read_csv function.

In [2]:
# read data
df = pd.read_csv('data/data_full.csv')

__Inspecting the data__

The first step is to visually inspect the new dataset. There are multiple ways to acheive this:
* The easiest way is to fetch first 5 rows is using DataFrame.head(), here df.head().
* Alternatively we can fetch the last 5 rows using DataFrame.tail(), here df.tail().

__NOTE:__ 

For both the above methods we can add a parameter inside the parenthesis '()' to specify how many rows we want to display, thus we can inspect the data.

In [3]:
# check shape of the given data
df.shape

(34857, 21)

We can observe here the number of rows and columns on the dataset.